##  Predictive Analytics Project

Yahoo Finance API

In [3]:
# pip install yfinance

In [2]:
# Importing necessary library
import yfinance as yf

# Define the ticker symbol
tickerSymbol = 'AAPL'

# Get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

# Get the historical prices for this ticker
tickerDf = tickerData.history(period='1d', start='2022-01-01', end='2023-10-01')

# See your data
print(tickerDf)

                                 Open        High         Low       Close  \
Date                                                                        
2022-01-03 00:00:00-05:00  176.052746  181.052279  175.933951  180.190964   
2022-01-04 00:00:00-05:00  180.804793  181.111693  177.329863  177.904068   
2022-01-05 00:00:00-05:00  177.814959  178.369360  172.894629  173.171829   
2022-01-06 00:00:00-05:00  170.974038  173.548059  169.924634  170.281036   
2022-01-07 00:00:00-05:00  171.162116  172.399623  169.320704  170.449310   
...                               ...         ...         ...         ...   
2023-09-25 00:00:00-04:00  174.199997  176.970001  174.149994  176.080002   
2023-09-26 00:00:00-04:00  174.820007  175.199997  171.660004  171.960007   
2023-09-27 00:00:00-04:00  172.619995  173.039993  169.050003  170.429993   
2023-09-28 00:00:00-04:00  169.339996  172.029999  167.619995  170.690002   
2023-09-29 00:00:00-04:00  172.020004  173.070007  170.339996  171.210007   

###### Alpha Vantage API:

In [3]:
# # Importing necessary library
# import requests

# # Define the API key and endpoint
# api_key = 'your_api_key_here'
# symbol = 'AAPL'
# base_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}'

# # Make the API call
# response = requests.get(base_url)
# data = response.json()

# # See your data
# print(data)

In [6]:
tickerDf.sample(3)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-01-20 00:00:00-05:00,134.705278,137.433643,133.649784,137.284271,80223600,0.0,0.0
2023-08-29 00:00:00-04:00,179.699997,184.899994,179.500000,184.119995,53003900,0.0,0.0
2022-09-21 00:00:00-04:00,156.412108,157.803861,152.694174,152.813461,101696800,0.0,0.0
